# Mini Projdct (최익창)


## 주제 : 전자공시시스템 정기 보고서(사업/반기/분기보고서) 크롤링
- 대상기간 : 2018.01.01~2018.12.20
- 보고서종류 : 사업보고서, 분기보고서, 반기보고서
- domain : https://dart.fss.or.kr
  - stock_name : 삼성전자
  - area : 연결재무제표 > 연결현금흐름표 테이블의 모든 내용

### 동기 : 관심기업의 공시 재무정보 추출을 통한 투자 기초자료 DB화

### 사용한 라이브러리
- selenium

In [70]:
# Code Start

In [71]:
# import module
from selenium import webdriver
import pandas as pd


# execute chromedriver
ch_driver = webdriver.Chrome("C://Python37/chromedriver.exe")

In [72]:
# open domain webpage
domain = "https://dart.fss.or.kr"
ch_driver.get(domain)

In [73]:
# input the target stock name in search box
stock_name = "삼성전자"
input_box = ch_driver.find_element_by_id("textCrpNm")
input_box.send_keys(stock_name)

In [74]:
# 정기공시 유형모음 버튼 확장
ch_driver.find_element_by_id("publicTypeButton_01").click()

In [75]:
# 사업보고서(publicType1), 반기보고서(publicType2), 분기보고서 선택(publicType3)
for i in range(1, 3+1):
    element = ch_driver.find_element_by_id("publicType{report_num}".format(report_num=i))
    element.click()

In [76]:
# set start_date
start_date = 20180101
start_date_box = ch_driver.find_element_by_id("startDate")
start_date_box.clear()
start_date_box.send_keys(start_date)

In [77]:
# set end_date
end_date = 20181220
end_date_box = ch_driver.find_element_by_id("endDate")
end_date_box.clear()
end_date_box.send_keys(end_date)

In [78]:
# click the search button
search_btn_xpath = '//*[@id="searchForm"]/fieldset/p[4]/input'
search_btn = ch_driver.find_element_by_xpath(search_btn_xpath)
search_btn.click()

In [79]:
# get the report elements
articles_xpath = '//*[@id="listContents"]/div[1]/table/tbody/tr'
article_elements = ch_driver.find_elements_by_xpath(articles_xpath)

In [80]:
# extract report urls from report elements
report_links = []
for content in article_elements:
    print(content.find_elements_by_tag_name('a')[-1].get_attribute('href'))
    report_links.append(content.find_elements_by_tag_name('a')[-1].get_attribute('href'))

http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20181114001530
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180814001113
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180515001699
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402005019


<img align="left" src="./ref/mini_pj10.png" width="600">

In [81]:
# set result report repos as number of links
result_list = [[]*i for i in range(len(report_links))]

# declear funtion(num_converter)
def num_converter(x):
    return int(x.replace(",", "").replace(")", "").replace("(", "-"))

# get the target data from each report
for link in report_links:
    # get the report index
    link_index = report_links.index(link)
    # get the report @ link
    ch_driver.get(link) 
    # 연결재무제표 페이지로 접근
    ch_driver.find_element_by_partial_link_text("2. 연결재무제표").click()
    # 연결재무제표 값을 갖고 있는 iframe으로 접근
    iframe = ch_driver.find_element_by_xpath('//*[@id="ifrm"]')
    ch_driver.switch_to.frame(iframe)
    # 현금흐름표 헤더 element 및 text 추출
    cashflow_header_xpath = '/html/body/table[not(@class="nb")][position()=last()]/thead/tr/th'
    cashflow_header = ch_driver.find_elements_by_xpath(cashflow_header_xpath)
    header_list = []
    for header in cashflow_header:
        header_list.append(header.text)
    # 현금흐름표 본문 추출(1) : tr element 추출
    cashflow_infos_xpath = '/html/body/table[not(@class="nb")][position()=last()]/tbody/tr'
    cashflow_infos = ch_driver.find_elements_by_xpath(cashflow_infos_xpath)
    cashflow_info_list = []
    # 현금흐름표 본문 추출(2) : tr 내 td element 추출
    for item in cashflow_infos:
        cashflow_infos_tds = item.find_elements_by_tag_name("td")
        temp_tds_list = []
    # 현금흐름표 본문 추출(3) : td element의 text 추출 > 값이 없으면 0으로 처리, 숫자형 문자는 int로 변환    
        for td in cashflow_infos_tds:
            temp_value = "0" if td.text == '' else td.text
            temp_tds_list.append(temp_value)
        temp_tds_list[1:] = list(map(num_converter, temp_tds_list[1:]))
        cashflow_info_list.append(temp_tds_list) 
    result_list[link_index].append(header_list)
    result_list[link_index].append(cashflow_info_list)

<img align="left" src="./ref/mini_pj11.png" width="600">

In [86]:
# result_list를 DataFrame으로 변환하여 df_list에 각각 적재
df_list  = []
for i in range(len(result_list)):
    temp_df = pd.DataFrame(data=result_list[i][1], columns=result_list[i][0])
    df_list.append(temp_df)

In [87]:
# index setting
for i, df in enumerate(df_list):
    df.set_index(keys=df.iloc[:,0], inplace=True)
    df = df.iloc[:, 1:]
    df_list[i] = df

In [88]:
# DataFrame Merge
df_merged = df_list[0]
for i in range(1, len(df_list)):
    df_merged = pd.merge(df_merged, df_list[i], how='outer', left_index=True, right_index=True)

In [89]:
# 보고서 merge를 통한 null값 확인
df_merged.isnull().sum()

제 50 기 3분기    14
제 49 기 3분기    14
제 50 기 반기     14
제 49 기 반기     14
제 50 기 1분기    14
제 49 기 1분기    14
제 49 기        15
제 48 기        15
제 47 기        15
dtype: int64

In [91]:
df_merged.head(10)

,제 50 기 3분기,제 49 기 3분기,제 50 기 반기,제 49 기 반기,제 50 기 1분기,제 49 기 1분기,제 49 기,제 48 기,제 47 기
,,,,,,,,,
관계기업 및 공동기업 투자의 처분,148.0,356608.0,148.0,356087.0,148.0,76522.0,355926.0,2280203.0,278009.0
관계기업 및 공동기업 투자의 취득,-17564.0,-19510.0,-17137.0,-12850.0,-17137.0,-4160.0,-25293.0,-84306.0,-137917.0
기말 현금 및 현금성자산,33088093.0,30788226.0,31362222.0,29007086.0,32303752.0,27562938.0,30545130.0,32111442.0,22636744.0
기초 현금 및 현금성자산,30545130.0,32111442.0,30545130.0,32111442.0,30545130.0,32111442.0,32111442.0,22636744.0,16840766.0
기타포괄손익-공정가치금융자산의 처분,9789.0,0.0,2230.0,0.0,2083.0,0.0,NaN,NaN,NaN
기타포괄손익-공정가치금융자산의 취득,-417447.0,0.0,-317963.0,0.0,-31911.0,0.0,NaN,NaN,NaN
단기금융상품의 순(증가)감소,-9026701.0,11124497.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
단기금융상품의 순감소,NaN,NaN,1212089.0,14228691.0,NaN,NaN,NaN,NaN,NaN
단기금융상품의 순감소(증가),NaN,NaN,NaN,NaN,3472296.0,10082810.0,387627.0,-6780610.0,-5762783.0


## 느낀 점

### 어려웠던 점
1. 페이지마다 크롤링 대상 xpath가 일정하지 않은 경우 어떻게 원하는 데이터를 끌고 올 수 있는가? 패턴찾기가 어려웠음
2. 패턴을 찾아도 어떻게 표현할 수 있는지 알아가는 과정이 쉽지 않았음

### 보완할 부분
1. selenium만 사용하여 크롤링을 하다 보니, 속도 이슈가 발생한다.
2. 보완하기 위해 api를 통한 크롤링을 검토해보는 것이 좋겠다.
   - 수많은 종목의 모든 보고서데이터를 DB화 하기 위해서..

### 소감
1. selenium만 사용하다보니 하나하나 클릭하며 목표에 접근할 때의 성취감
2. 구조가 서로 다른 여러문서를 크롤링을 통한 패턴찾기의 즐거움(?)
3. Trial and Error, Googling의 즐거움(?)

---
## 특이사항
- 종목을 입력하여 게시글을 검색한 뒤, 클릭하면 팝업창 생성

<img src="./ref/mini_pj01.png" width="800">

---
- 팝업창에는 목차와 세부내용이 표시됨
  - 세부내용은 iframe으로 구성되어 있음
  - 목차에서 다른 내용을 선택하더라도 팝업창에 표시된 url은 변동 없음

<img src="./ref/mini_pj02.png" width="800">

- API를 사용할 수 있는지 체크해보자.
- docs에 요소가 하나 감지되고, 해당내용은 바로 내가 원하는 페이지이다.

<img src="./ref/mini_pj06.png" width="800">

- 그럼 해당 요소의 URL을 복사하고

<img src="./ref/mini_pj07.png" width="800">

- postman에 대입하여 원하는 내용이 제약없이 반환되는지 확인해보자.<br>
  → 잘 반환되는 것을 확인할 수 있다.
  - 또한, 총 6개의 파라미터가 존재하는 것을 확인할 수 있다. <br>
    (rcpNo, dcmNo, eleId, offset, length, dtd)<br>

<img src="./ref/mini_pj08.png" width="800">

- 그렇다면 페이지 구성을 위해 사용하는 파라미터는 어디에서 오는 것인가?..
- 혹시 팝업창을 띄울때 불러오는 Element에 있지는 않은가?.. 확인해보자.

# Find를 사용하여 찾아보니..
- 팝업창을 띄울 때 넘어오는 html구문 속에 이미
- 모든 목차의 내용을 끌고올 수 있도록 key, value값이 셋팅되어 있다.

<img src="./ref/mini_pj09.png" width="800">

- network에서 확인한 target element의 link url과 비교해보자. (내용이 동일하다.)
- http://dart.fss.or.kr/report/viewer.do?rcpNo=20181114001530&dcmNo=6382016&eleId=13&offset=594590&length=94473&dtd=dart3.xsd <br><br>
- selenium으로 진행했을 때의 단점(페이지 하나씩 브라우징에 따른 속도 이슈) 감안 시 <br>
  다량의 데이터를 crawling한다면 api를 이용하는 방법이 더 적합한 것으로 보인다.